# Import Libraries

In [73]:
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split

# Load data

Columns like PassengerID, Name, Ticket don't seem to be a good feature. Cabin is usually `NaN`. It's better to remove these columns from the data.

In [74]:
# Load the dataset
data = pd.read_csv('train.csv')

# Drop some columns and rows
data = data.drop(columns=["PassengerId", "Name", "Ticket", "Cabin"])
data['Age'].fillna(data['Age'].mean(), inplace=True)
data['Embarked'].fillna(data['Embarked'].mode()[0], inplace=True)

print(f"Number of rows {data.shape[0]}")
data.head(5)

Number of rows 891


,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


# Add some features

I add 3 more features to the features.
- **FamilySize**: Tells the size of the family which is the sum of `SibSp` and `Parch`. 
- **HasSomebody**: It is true (=1) if the person is so young (`Age` < 22) or so old (`Age` > 60) and does not have any family (=Solo).
- **Ability**: Tells about amount of physical ability to escape or save themselves.

In [75]:
# Create a new feature that represents the size of the passenger's family
data['FamilySize'] = data['SibSp'] + data['Parch']

# Create a new feature that represent a person need somebody to take care of or not
data['HasSomebody'] = pd.cut(data['Age'], bins=[-1, 22, 60, 200], labels=[0, 1, 0], ordered=False).astype('int64')
data['HasSomebody'] = data['HasSomebody'] + data['FamilySize']
data['HasSomebody'] = pd.cut(data['HasSomebody'], bins=[-1, 0, 300], labels=[0, 1])

# Create a new feature that represents the ability based on age
data['Ability'] = pd.cut(data['Age'], bins=[-1, 4, 15, 40, 60, 200], labels=['Low', 'Medium', 'High', 'Medium', 'Low'], ordered=False)
data['FamilySize'] = pd.cut(data['FamilySize'], bins=[-1, 0, 3, 7, 20], labels=['Solo', 'Small', 'Medium', 'Large'])

# Convert categorical features to numerical
data = pd.get_dummies(data, columns=['Sex', 'Embarked', 'FamilySize', 'Ability'])

data.head(5)

,Survived,Pclass,Age,SibSp,Parch,Fare,HasSomebody,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S,FamilySize_Solo,FamilySize_Small,FamilySize_Medium,FamilySize_Large,Ability_High,Ability_Low,Ability_Medium
0,0,3,22.0,1,0,7.2500,1,0,1,0,0,1,0,1,0,0,1,0,0
1,1,1,38.0,1,0,71.2833,1,1,0,1,0,0,0,1,0,0,1,0,0
2,1,3,26.0,0,0,7.9250,1,1,0,0,0,1,1,0,0,0,1,0,0
3,1,1,35.0,1,0,53.1000,1,1,0,0,0,1,0,1,0,0,1,0,0
4,0,3,35.0,0,0,8.0500,1,0,1,0,0,1,1,0,0,0,1,0,0


# Provide the train and test data

I consider 30% of the train test for test data. There are not many records to train. As I know, in these cases, it's better to dedicate more than 20% of our train data to the test data. 

In [76]:
X = data.drop('Survived', axis=1)
y = data['Survived']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Build the Model and Training

In [77]:
# Build the model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/50
20/20 [==============================] - 1s 25ms/step - loss: 0.6976 - accuracy: 0.6629 - val_loss: 0.6356 - val_accuracy: 0.6903
Epoch 2/50
20/20 [==============================] - 0s 7ms/step - loss: 0.6122 - accuracy: 0.6918 - val_loss: 0.5635 - val_accuracy: 0.6866
Epoch 3/50
20/20 [==============================] - 0s 9ms/step - loss: 0.5580 - accuracy: 0.7095 - val_loss: 0.5384 - val_accuracy: 0.7351
Epoch 4/50
20/20 [==============================] - 0s 7ms/step - loss: 0.5244 - accuracy: 0.7560 - val_loss: 0.5512 - val_accuracy: 0.7090
Epoch 5/50
20/20 [==============================] - 0s 7ms/step - loss: 0.5103 - accuracy: 0.7512 - val_loss: 0.4979 - val_accuracy: 0.7612
Epoch 6/50
20/20 [==============================] - 0s 8ms/step - loss: 0.4977 - accuracy: 0.7865 - val_loss: 0.4985 - val_accuracy: 0.7687
Epoch 7/50
20/20 [==============================] - 0s 7ms/step - loss: 0.4830 - accuracy: 0.7865 - val_loss: 0.4776 - val_accuracy: 0.7724
Epoch 8/50
20/20 [=

# Evaluation

In [78]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Accuracy: {accuracy}")

9/9 [==============================] - 0s 3ms/step - loss: 0.4289 - accuracy: 0.8022
Accuracy: 0.8022388219833374


# Judge the unseen test data

In [79]:
# Load the dataset
data = pd.read_csv('test.csv')
backup_data = data

# Drop some columns and rows
data = data.drop(columns=["PassengerId", "Name", "Ticket", "Cabin"])
data['Age'].fillna(data['Age'].mean(), inplace=True)
data['Embarked'].fillna(data['Embarked'].mode()[0], inplace=True)

print(f"Number of rows {data.shape[0]}")

# Create a new feature that represents the size of the passenger's family
data['FamilySize'] = data['SibSp'] + data['Parch']

# Create a new feature that represent a person need somebody to take care of or not
data['HasSomebody'] = pd.cut(data['Age'], bins=[-1, 22, 60, 200], labels=[0, 1, 0], ordered=False).astype('int64')
data['HasSomebody'] = data['HasSomebody'] + data['FamilySize']
data['HasSomebody'] = pd.cut(data['HasSomebody'], bins=[-1, 0, 300], labels=[0, 1])

# Create a new feature that represents the ability based on age
data['Ability'] = pd.cut(data['Age'], bins=[-1, 4, 15, 40, 60, 200], labels=['Low', 'Medium', 'High', 'Medium', 'Low'], ordered=False)
data['FamilySize'] = pd.cut(data['FamilySize'], bins=[-1, 0, 3, 7, 20], labels=['Solo', 'Small', 'Medium', 'Large'])

# Convert categorical features to numerical
data = pd.get_dummies(data, columns=['Sex', 'Embarked', 'FamilySize', 'Ability'])

Number of rows 418


In [80]:
y_pred = model.predict(data)
y_pred_binary = (y_pred >= 0.5).astype(int)
backup_data['Survived'] = y_pred_binary

14/14 [==============================] - 0s 2ms/step


In [81]:
# Create the output file
output = pd.DataFrame()
output['PassengerId'] = backup_data['PassengerId']
output['Survived'] = backup_data['Survived']
output.to_csv("gender_submission.csv", header=True)